In [1]:
import json
import pandas as pd
import csv
import glob
import sys

# Create the xml files for java parser

In [ ]:
file = 'Taxiarchis-data-four_categories_part.csv'

df =pd.read_csv(file, delimiter='\t',error_bad_lines=False)

In [ ]:
df.columns

In [ ]:
df.Categories.unique()

In [ ]:
df_cat = df[['DOI_arxiv','Categories']]

In [ ]:
df_cat.to_csv('final_categories.csv',index = False)

In [ ]:
df.shape

In [ ]:
import xml.etree.ElementTree as ET 

folder = 'C:\\Users\\papakostas-ioannit\\Desktop\\Python_Four_Categories\\xmls'

for index, row in df.iterrows():
    tree = ET.fromstring(row.content)
    
    with open(folder+'\\'+str(index)+".xml","wb") as file:
            file.write(ET.tostring(tree))
    print(row.DOI_arxiv)

# Create final csv with Sections

In [130]:
VersionTwo = 'four_categories_s2.csv'
VersionThree = 'four_categories_s3.csv'
VersionFour = 'four_categories_s4.csv'

In [131]:
df_categories.columns

NameError: name 'df_categories' is not defined

In [135]:
df_categories = pd.read_csv('final_categories.csv')
df_categories.columns = ['DOI', 'Categories']
df_categories.head()

,DOI,Categories
0,10.1016/j.aop.2008.03.005,Condensed Matter - Statistical Mechanics
1,10.1016/j.carbon.2016.10.063,Condensed Matter - Materials Science
2,10.1016/j.gca.2011.11.040,Condensed Matter - Statistical Mechanics
3,10.1016/j.jcp.2013.06.036,Condensed Matter - Materials Science
4,10.1016/j.jcrysgro.2006.11.214,Condensed Matter - Materials Science


In [132]:
#Maximize ram point limitation
maxInt = sys.maxsize 
decrement = True

while decrement:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

In [133]:
def createDataframes(filename):
    '''
    Creates a Dataframe with sections as a json file 
    and DOI and file name.
    '''
    
    notIncludeSections = ['Graphical abstract','Highlights', 'Keywords','PACS','References','Acknowledgments']
    headers = ['DOI','xml_name','Sections','Names']
    temp = []
    Sections = []
    flag = 0


    with open(filename,'r') as articles:

            reader = csv.reader(articles,delimiter='\t')

            for count,row in enumerate(reader):

                if len(row)>8:
                    data = dict()
                    
                    for section in row[2:-1]:
                        result = section.split(' ================== ')

                        if  result[0] not in notIncludeSections: 
                            data[result[0]] = result[1]
                        else: 
                            continue

                    if len(data) == 0:
                        continue
                    else:
                        json_data = json.dumps(data)
                        Sections.append([row[0],row[1], json_data, list(data.keys())])

    return pd.DataFrame(Sections, columns=headers)

    

In [14]:
file = "four_categories_s2.csv"
df = createDataframes(file)


In [32]:
df.head()

,DOI,xml_name,Sections,Names
0,10.1016/j.jnucmat.2011.08.041,7.xml,"{""Summary"": ""In summary, we have studied the s...","[Summary, Thermodynamic properties, Difference..."
1,10.1016/j.jssc.2014.08.004,8.xml,"{""Density derived electrostatic and chemical (...",[Density derived electrostatic and chemical (D...
2,10.1016/S0039-6028(01)01214-6,15.xml,"{""O(1\u00d71)/Ru(0001) and RuO2(110) surface s...",[O(1×1)/Ru(0001) and RuO2(110) surface structu...
3,10.1016/j.nimb.2006.12.036,17.xml,"{""Experimental details"": ""The studies performe...","[Experimental details, Acknowledgement, Conclu..."
4,10.1016/j.jmmm.2008.07.002,21.xml,"{""Summary"": ""We have carried out a comparative...","[Summary, MRPT in [110] and [001] films, Tempe..."


In [68]:
df['Names'][47]

['Experimental details',
 'Systems preparation and characterization',
 'Introduction',
 'Acknowledgements',
 'Abstract',
 'STS and DFT of gr/O/Ru(0001)',
 'Results and discussions',
 'ARPES and DFT of gr/Ir(111) and gr/Cu/Ir(111)',
 'Conclusions']

In [83]:
js_test = df['Sections'][47]
type(js_test)

str

In [136]:
#COMBINE All the files

files=[VersionTwo, VersionThree, VersionFour]
dataframes = []
for file in files:
    dataframes.append(createDataframes(file))

df_sections = pd.concat(dataframes) #Concatinate all dataframes
# df_categories = pd.read_csv('final_categories.csv')

df = pd.merge(df_sections,df_categories, on='DOI',how="inner") #Merge the sections with the categories based on DOI


In [166]:
match = 'Condensed Matter - Strongly Correlated Electrons'
df_strongly = df[df['Categories'] == match].reset_index()

df_strongly.head()

,index,DOI,xml_name,Sections,Names,Categories
0,0,10.1016/j.jnucmat.2011.08.041,7.xml,"{""Summary"": ""In summary, we have studied the s...","[Summary, Thermodynamic properties, Difference...",Condensed Matter - Strongly Correlated Electrons
1,3,10.1016/j.nimb.2006.12.036,17.xml,"{""Experimental details"": ""The studies performe...","[Experimental details, Acknowledgement, Conclu...",Condensed Matter - Strongly Correlated Electrons
2,5,10.1016/S0921-4526(99)01837-2,42.xml,"{""(TMTCF)2X"": ""Since the localized state along...","[(TMTCF)2X, HF calculations, ?-(ET)2X, Abstrac...",Condensed Matter - Strongly Correlated Electrons
3,8,10.1016/j.aop.2006.04.007,52.xml,"{""Conclusion"": ""To conclude, we have shown tha...","[Conclusion, Flowgrams for the short-range mod...",Condensed Matter - Strongly Correlated Electrons
4,31,10.1016/S0550-3213(03)00379-1,154.xml,"{""Model and formalism"": ""2.1SU(N)-invariant d-...","[Model and formalism, Introduction, Contact wi...",Condensed Matter - Strongly Correlated Electrons


In [180]:
df_strongly['Names'][14]

['Full Hamiltonian',
 'Discussion, open questions, and conclusions',
 'Small-angle neutron scattering',
 'Models and parameters',
 'PDF techniques',
 'Double-exchange is not enough',
 'Introduction',
 'Possible mixed-phase tendencies in nonmanganite compounds',
 'Influence of long-range Coulomb interactions',
 'Related theoretical work on electronic phase separation applied to manganites',
 'Mean-field approximation for H?',
 'La1?xSrxMnO3 at low density',
 'Quantum localized spins',
 'Ferromagnetism due to a large Hund coupling',
 'La1?xCaxMnO3 at density 0.0?x<0.5',
 'Electron–phonon coupling',
 'Photoemission',
 'Double-layer compound',
 'X-ray absorption, transport and magnetism',
 'Scanning tunneling spectroscopy',
 'Importance of tolerance factor',
 'Bi-stripe structure at x>0.5',
 'More recent theories',
 'JH=? limit',
 'Studies with high pressure',
 'Neutron scattering',
 'Charge ordering at x=0.5 and the CE-state',
 'Coulomb interactions',
 'Charge order at x<0.5',
 'Hall effe

In [140]:
df[['Categories','DOI']].groupby('Categories').count()

,DOI
Categories,
Condensed Matter - Materials Science,2375
Condensed Matter - Mesoscale and Nanoscale Physics,1460
Condensed Matter - Statistical Mechanics,2061
Condensed Matter - Strongly Correlated Electrons,1020


# Create 4 features

In [152]:
import json
#Final Deployment!!!!
l = []
headers = ['doc_id', 'Category','DOI', 'Section','Sec_Name'] 


for index, row in df.iterrows():
    
    flag = 0
    
    temp = []
    #Load json structure - Extract sections
    try_js_sec = json.loads(row.Sections)
    sec = list(try_js_sec.keys())
    
    if (str('Abstract') in sec) and (str('Introduction') in sec):

        sec.remove('Abstract')
        sec.remove('Introduction')

        temp.append([str(index)+'_'+str(flag), row.Categories, row.DOI, try_js_sec['Abstract'], 'Abstract'])
        flag += 1
        temp.append([str(index)+'_'+str(flag), row.Categories, row.DOI,try_js_sec['Introduction'], 'Introduction'])
        flag += 1
#         print(sec)

    else:
        pass


    conlu_sections=['Conclusions','Conclusion' ,"Discussion and conclusion", 'Concluding remarks',\
                    'Results and discussion','Results and discussions','Summary and discussion', \
                    'Summary and discussions', 'Numerical performance and concluding discussion',\
                    'Concluding remarks', 'Discussion', 'Discussions', 'Summary']
    fla2=0
    for i in conlu_sections:
#         print(i)
        if i in sec:
#             print(sec)
            temp.append([str(index)+'_'+str(flag), row.Categories, row.DOI,try_js_sec[i], 'Conclusions'])
            flag += 1
            break


    #IN-BETWEEN    
    text = str()  
    for s in sec:

        text+= try_js_sec[s]


    temp.append([str(index)+'_'+str(flag), row.Categories, row.DOI, text, 'In_Between']) 

    if len(temp)==4:
        for r in temp:
            l.append(r)

test_df = pd.DataFrame(l, columns=headers)
test_df.head()
       

,doc_id,Category,DOI,Section,Sec_Name
0,0_0,Condensed Matter - Strongly Correlated Electrons,10.1016/j.jnucmat.2011.08.041,We systematically investigate the electronic s...,Abstract
1,0_1,Condensed Matter - Strongly Correlated Electrons,10.1016/j.jnucmat.2011.08.041,Plutonium-based materials have been extensivel...,Introduction
2,0_2,Condensed Matter - Strongly Correlated Electrons,10.1016/j.jnucmat.2011.08.041,3.1Electronic structure of PuO2In order to inv...,Conclusions
3,0_3,Condensed Matter - Strongly Correlated Electrons,10.1016/j.jnucmat.2011.08.041,"In summary, we have studied the structural, el...",In_Between
4,1_0,Condensed Matter - Materials Science,10.1016/j.jssc.2014.08.004,Atomic partial charges are parameters of key i...,Abstract


In [182]:
test_df.to_csv('Section_4_4_Ensemble.csv', quoting=csv.QUOTE_NONE, index=False, escapechar='\\',sep ='\t')

In [181]:
test_df.shape

(22288, 5)

In [3]:
6676/4

1669.0

In [193]:
drop = test_df[['Category', 'DOI']].drop_duplicates().reset_index()
drop.shape

(5552, 3)

In [197]:
drop = drop[['DOI','Category']]

In [198]:
drop.head()

,DOI,Category
0,10.1016/j.jnucmat.2011.08.041,Condensed Matter - Strongly Correlated Electrons
1,10.1016/j.jssc.2014.08.004,Condensed Matter - Materials Science
2,10.1016/S0039-6028(01)01214-6,Condensed Matter - Materials Science
3,10.1016/j.nimb.2006.12.036,Condensed Matter - Strongly Correlated Electrons
4,10.1016/j.jmmm.2008.07.002,Condensed Matter - Materials Science


In [199]:
drop.to_csv('doi_categories_4_4_features.csv', quoting=csv.QUOTE_NONE, index=False, escapechar='\\',sep ='\t')

In [ ]:
27664-22288

In [155]:
smpl = test_df[['Category','DOI']].groupby('Category').count()
smpl['DOI'].apply(lambda x : x/4)


Category
Condensed Matter - Materials Science                  2095.0
Condensed Matter - Mesoscale and Nanoscale Physics    1158.0
Condensed Matter - Statistical Mechanics              1540.0
Condensed Matter - Strongly Correlated Electrons       779.0
Name: DOI, dtype: float64

In [194]:
2095+1158+1540+779

5572

In [ ]:
df[['DOI', 'Sections', 'Categories']].head(10)

In [15]:
df[['DOI', 'Categories']].to_csv('DOI_Categories_4.csv', quoting=csv.QUOTE_NONE, index=False, escapechar='\\',sep ='\t')

In [ ]:
df[[ 'Categories','DOI','Sections']].to_csv('Sections_2.csv', quoting=csv.QUOTE_NONE, index=False, escapechar='\\',sep ='\t')

# Create the whole document

In [21]:
def createDataframes_fullDocument(filename):
    '''
    Creates a Dataframe with sections as a json file 
    and DOI and file name.
    '''
    
    notIncludeSections = ['Graphical abstract','Highlights', 'Keywords','PACS','References','Acknowledgments']
    headers = ['DOI','xml_name','Full_Document']
    temp = []
    Sections = []
    flag = 0


    with open(filename,'r') as articles:

            reader = csv.reader(articles,delimiter='\t')

            for count,row in enumerate(reader):

                if len(row)>8:
                    data = str()
                    name_sec = 'Section_'
                    for count1, section in enumerate(row[2:-1]):
                        result = section.split(' ================== ')

                        if  result[0] not in notIncludeSections: 
#                             data[name_sec+str(count1)] = result[1]
                            data+=result[1] 
                        else: 
                            continue

                    if len(data) == 0:
                        continue
                    else:
#                         json_data = json.dumps(data)
                        Sections.append([row[0],row[1], data])

    return pd.DataFrame(Sections, columns=headers)

    
# test = createDataframes_fullDocument(VersionFour)
# test.head()

In [24]:
files=[VersionTwo, VersionThree, VersionFour]
dataframes = []
for file in files:
    dataframes.append(createDataframes_fullDocument(file))

df_fullDocument = pd.concat(dataframes) #Concatinate all dataframes
# df_categories = pd.read_csv('final_categories.csv')

df_full = pd.merge(df_fullDocument, df_categories, on='DOI',how="inner") #Merge the sections with the categories based on DOI

In [25]:
df_full.head()

,DOI,xml_name,Full_Document,Categories
0,10.1016/j.jnucmat.2011.08.041,7.xml,We systematically investigate the electronic s...,Condensed Matter - Strongly Correlated Electrons
1,10.1016/j.jssc.2014.08.004,8.xml,Atomic partial charges are parameters of key i...,Condensed Matter - Materials Science
2,10.1016/S0039-6028(01)01214-6,15.xml,We present density-functional theory calculati...,Condensed Matter - Materials Science
3,10.1016/j.nimb.2006.12.036,17.xml,In this work we have studied systematically th...,Condensed Matter - Strongly Correlated Electrons
4,10.1016/j.jmmm.2008.07.002,21.xml,The isothermal magnetoresistance [R(?)] of [00...,Condensed Matter - Materials Science


In [ ]:
df_full.shape

In [ ]:
df_full.Full_Document[1]

In [26]:
df_full[[ 'Categories','DOI','Full_Document']].to_csv('FullDocument_4.csv', quoting=csv.QUOTE_NONE, index=False, escapechar='\\',sep ='\t')

# Per section

In [ ]:
# def createDataframes(filename):
#     '''
#     Creates a Dataframe with sections as a json file 
#     and DOI and file name.
#     '''
    
#     notIncludeSections = ['Graphical abstract','Highlights', 'Keywords','PACS','References','Acknowledgments']
#     headers = ['DOI','xml_name','Sections']
#     temp = []
#     Sections = []
#     flag = 0


#     with open(filename,'r') as articles:

#             reader = csv.reader(articles,delimiter='\t')

#             for count,row in enumerate(reader):

#                 if len(row)>4:
#                     data = dict()
                    
#                     for section in row[2:-1]:
#                         result = section.split(' ================== ')

#                         if  result[0] not in notIncludeSections: 
#                             data[result[0]] = result[1]
#                         else: 
#                             continue

#                     if len(data) == 0:
#                         continue
#                     else:
#                         json_data = json.dumps(data)
#                         Sections.append([row[0],row[1], json_data])

#     yield pd.DataFrame(Sections, columns=headers)


In [16]:
df.head()

,DOI,xml_name,Sections,Categories
0,10.1016/j.jnucmat.2011.08.041,7.xml,"{""Electronic structure of ?-Pu2O3"": ""?-Pu2O3 i...",Condensed Matter - Strongly Correlated Electrons
1,10.1016/j.jssc.2014.08.004,8.xml,"{""Density derived electrostatic and chemical (...",Condensed Matter - Materials Science
2,10.1016/S0039-6028(01)01214-6,15.xml,"{""XPS fingerprinting of the oxidation sequence...",Condensed Matter - Materials Science
3,10.1016/j.nimb.2006.12.036,17.xml,"{""Broad irradiation"": ""The temperature depende...",Condensed Matter - Strongly Correlated Electrons
4,10.1016/j.jmmm.2008.07.002,21.xml,"{""Summary"": ""We have carried out a comparative...",Condensed Matter - Materials Science


In [32]:
import json

l = []
headers = ['doc_id', 'Category','DOI', 'Section'] 
for index, row in df.iterrows():

    js = json.loads(row.Sections)
    for sec_id,i in enumerate(js.keys()):
        
        #Filter Out section containing small text length, might be parsed wrong
        if len(js[i]) <100:
            continue
        else:
            l.append([str(index)+'_'+str(sec_id), row.Categories,row.DOI, js[i]])

test_df = pd.DataFrame(l, columns=headers)
test_df.head()

,doc_id,Category,DOI,Section
0,0_0,Condensed Matter - Strongly Correlated Electrons,10.1016/j.jnucmat.2011.08.041,?-Pu2O3 in body-centered cubic structure can b...
1,0_1,Condensed Matter - Strongly Correlated Electrons,10.1016/j.jnucmat.2011.08.041,To know the thermodynamic properties from PuO2...
2,0_2,Condensed Matter - Strongly Correlated Electrons,10.1016/j.jnucmat.2011.08.041,Our first-principles calculations are based on...
3,0_3,Condensed Matter - Strongly Correlated Electrons,10.1016/j.jnucmat.2011.08.041,In order to investigate the electronic and str...
4,0_4,Condensed Matter - Strongly Correlated Electrons,10.1016/j.jnucmat.2011.08.041,"In PuO2 and ?-Pu2O3, the ideal oxidation state..."


In [33]:
test_df.shape

(71553, 4)

In [ ]:
test_df.head(15)

In [34]:
test_df.to_csv('perSection_4_1.csv', quoting=csv.QUOTE_NONE, index=False, escapechar='\\',sep ='\t')

# Training -Test set create per Section

In [ ]:
from sklearn.cross_validation import train_test_split



In [ ]:
import pandas as pd

df = pd.read_csv('Sections_2.csv',sep='\t')
df.head()

In [ ]:
import numpy as np

msk = np.random.rand(len(df)) < 0.8


train = df[msk]
test = df[~msk]

print(train.shape)
print(test.shape)

In [ ]:
import json

def splitSections(datFrame):
    l = []
    headers = ['doc_id', 'Category','DOI', 'Section'] 
    for index, row in datFrame.iterrows():
        try:
            js = json.loads(row.Sections)
            for sec_id,i in enumerate(js.keys()):

                l.append([str(index)+'_'+str(sec_id), row.Categories,row.DOI, js[i]])
        except:

            continue

    return pd.DataFrame(l, columns=headers)

In [ ]:
df_train = splitSections(train)
df_train.shape

In [ ]:
df_train.head()

In [ ]:
df_test = splitSections(test)
df_test.shape

In [ ]:
import csv

df_train[[ 'doc_id','Category','DOI','Section']].to_csv('Train_Sections.csv', quoting=csv.QUOTE_NONE, index=False, escapechar='\\',sep ='\t')

In [ ]:
df_test[[ 'doc_id','Category','DOI','Section']].to_csv('Test_Sections.csv', quoting=csv.QUOTE_NONE, index=False, escapechar='\\',sep ='\t')

# Majority Vote

In [ ]:
df_te = pd.DataFrame( {'a':['A','A','B','B','B','C'], 'b':[1,2,5,5,4,6]})
df_te

In [ ]:
test_doc = [['id_1',1,'Section_1' ],
           ['id_1',1, 'Section_2'],
           ['id_1',0, 'Section_3'],
           ['id_1',2, 'Section_4'],
           ['id_1',1, 'Section_5'],
           ['id_1',0, 'Section_6'],
           ['id_2',1,'Section_1' ],
           ['id_2',1, 'Section_2'],
           ['id_2',0, 'Section_3'],
           ['id_2',0, 'Section_4'],
           ['id_2',0, 'Section_5'],
           ['id_2',0, 'Section_6'] ]

headers = ['doc_id', 'Category', 'Section'] 


df = pd.DataFrame(test_doc, columns=headers)
# df.doc_id.unique()
df

In [ ]:
df.to_csv('dummy_data.csv',index = False)

In [ ]:
def dataframe_for_majorityVote(datFrame):
    '''
    Transforms the dataframe for 
    majority vote function. 
    '''
#     df.doc_id.unique()
    
    
    return datFrame.groupby('doc_id')['Category'].apply(list)
    


In [ ]:
mV = pd.DataFrame(dataframe_for_majorityVote(df)).reset_index()

In [ ]:
mV

In [ ]:
def majority_vote(votes):
    '''
    xxx
    '''
    
    vote_counts ={}
    for vote in votes:
        if vote in vote_counts:
            vote_counts[vote]+=1
        else:
            vote_counts[vote] = 1
    print(vote_counts.values())
    return max(vote_counts, key=vote_counts.get)
    
    

In [ ]:
mV['Predicted'] = mV['Category'].apply(majority_vote)
mV

In [30]:
t = 'Supplementary data'


In [31]:
len(t)>100

False

In [35]:
ta = 'The second author gratefully acknowledges the support of project AVOZ10100520 of the Academy of Sciences of the Czech Republic.'

len(ta)

127

# 4 Feature Build

In [127]:
l = []
flag = 0
index = 0
sec = ['Experimental details',
 'Systems preparation and characterization',
 'Introduction',
 'Acknowledgements',
 'Abstract',
 'STS and DFT of gr/O/Ru(0001)',
 'Results and discussions',
 'ARPES and DFT of gr/Ir(111) and gr/Cu/Ir(111)',
 'Conclusions']

if (str('Abstract') in sec) and (str('Introduction') in sec):
    
    sec.remove('Abstract')
    sec.remove('Introduction')
    
    l.append([str(index)+'_'+str(flag), try_js_sec['Abstract'], 'Abstract'])
    flag += 1
    l.append([str(index)+'_'+str(flag), try_js_sec['Introduction'], 'Introduction'])
    flag += 1
    print(sec)
    
else:
    pass


conlu_sections=['Conclusions','Conclusion' ,"Discussion and conclusion", 'Concluding remarks',\
                'Results and discussion','Results and discussions','Summary and discussion', \
                'Summary and discussions', 'Numerical performance and concluding discussion',\
                'Concluding remarks', 'Discussion', 'Discussions', 'Summary']

for i in conlu_sections:
    print(i)
    if i in sec:
        print(sec)
        l.append([str(index)+'_'+str(flag), try_js_sec[i], 'Conclusions'])
        flag += 1
        break
        

#IN-BETWEEN    
text = str()  
for s in sec:
    
    text+= try_js_sec[s]
    
    
l.append([str(index)+'_'+str(flag), text, 'In_Between']) 


headers = ['doc_id', 'Section', 'Sec_Name'] 
test_df = pd.DataFrame(l, columns=headers)
test_df.head()

['Experimental details', 'Systems preparation and characterization', 'Acknowledgements', 'STS and DFT of gr/O/Ru(0001)', 'Results and discussions', 'ARPES and DFT of gr/Ir(111) and gr/Cu/Ir(111)', 'Conclusions']
Conclusions
['Experimental details', 'Systems preparation and characterization', 'Acknowledgements', 'STS and DFT of gr/O/Ru(0001)', 'Results and discussions', 'ARPES and DFT of gr/Ir(111) and gr/Cu/Ir(111)', 'Conclusions']


,doc_id,Section,Sec_Name
0,0_0,Intercalation of different species under graph...,Abstract
1,0_1,Intercalation compounds on the basis of differ...,Introduction
2,0_2,In the present manuscript we demonstrate the s...,Conclusions
3,0_3,Graphene in both experiments was prepared on h...,In_Between
